# Num. of people living in forests by municipality (Colombia)

### Load packages

In [1]:
import os
import ee
import geemap

In [2]:
ee.Initialize()
#ee.Authenticate()

### Set data parameters

In [3]:
# Selected country 
country = 'Colombia';
# Canopy cover percentage (e.g. 10%).
cc = ee.Number(25);
# Minimum forest area in pixels (e.g. 6 pixels, ~ 0.5 ha in this example).
pixels = ee.Number(6);
# Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = ee.Number(6);

## Forest cover data

In [4]:
#Load country features from Large Scale International Boundary (LSIB) dataset.
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
country_shp = countries.filter(ee.Filter.eq('country_na', ee.String(country)));

#Load GFC data for 2020
gfc = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')
#gfc = gfc.reproject(crs='EPSG:4326', scale=1000)
#gfc = gfc.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=10e10)

In [5]:
#Select tree cover
canopyCover = gfc.select(['treecover2000']) #Make areas below the canopy cover transparent
canopyCover = canopyCover.gte(cc).selfMask()
canopyCover = canopyCover.connectedPixelCount().gte(pixels).selfMask().clip(country_shp)
treecover = canopyCover

In [6]:
#Genrate tree cover loss images for every year
lossyear = gfc.select(['lossyear']).clip(country_shp)

loss_list = []
for i in range(0,21):
    loss = lossyear.lte(i)
    loss = treecover.mask(loss).selfMask()
    loss_list.append(loss)
    
    #Note: loss_list[0]= no loss in year 2000 (basline forest cover), 
    #loss_list[20]= pixels of tree cover loss in 2020    

In [7]:
#Generate tree cover images for every year
cover_in_years = []

for i in range(0,21):
    binary = loss_list[i].unmask(0)
    binary = binary.eq(0).And(treecover.eq(1)).selfMask()
    binary = binary.unmask(0). clip(country_shp)
        
    cover_in_years.append(binary)
    
    #Note: cover_in_years[0]= tree cover pixels in 2000...

## Population living in forests by municipalities

#### Import municipality shapefile

In [8]:
#geemap.ee_search()
municipios = ee.FeatureCollection('users/emileg1079/municipios')

### Visualize the data at the country level 

In [9]:
#Population for the country
cover = cover_in_years[0].selfMask()
gpw = ee.Image('CIESIN/GPWv4/population-count/2020').updateMask(cover)

#### Map layers

In [10]:
Map = geemap.Map()

raster_vis = {
  "max": 1000.0,
  "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
  ],
  "min": 0.0
}

Map.addLayer(cover, {'min': 0,'max': 1,  'palette': ['green']} ,'cover2000')
Map.addLayer(gpw, raster_vis, 'population_2020')

Map.setCenter(-68, 5, 6)
Map

Map(center=[5, -68], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

#### Count the total population living in forests in the country

In [11]:
# Load a polygon feature
polygon = municipios

# Reduce the population data within the polygon
population_count = gpw.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=polygon.geometry(),
  scale=1000,
  maxPixels=1e12
).getInfo()['population-count']

# Print the population count for people living in forests in year 2000
print('Population count:', population_count)

Population count: 30661415.48427544


#### Function for adding popultion by municipality

In [12]:
#Define function to aggregate raster data over a geometry
def mun_temp(region,var,agg_fun) :
    total = var.reduceRegion(
        reducer=agg_fun,
        geometry=region.geometry(),
        scale=1000,
        maxPixels=1e10
        );
    return region.set(total)

#### Loop over function over years for changing forests

In [22]:
#Loop that sums population over 1km2 pixels of forests (changing forests)
for y in ["2005","2010","2015","2020"] :
    
    #Load tree cover data
    n = int(y)-2000 
    cover = cover_in_years[n].selfMask()
    
    #Load grided population of the world data
    gpw = ee.Image(f'CIESIN/GPWv4/population-count/{y}').updateMask(cover)
    gpw_y_pop = gpw.select("population-count")
    
    #Excecute function 
    gpw_y_sum = municipios.map(lambda x: mun_temp(x, gpw_y_pop, ee.Reducer.sum()))

    #Save csv to drive
    task = ee.batch.Export.table.toDrive(collection=gpw_y_sum,
                                     folder="pop_in_forests_v3", # Nombre de la carpeta
                                     description=f"population_forest_{y}") # Nombre del archivo
                                     #fileFormat="shp") # Tipo de archivo
    task.start()

#### Population per municipality  GPW

In [21]:
#Loop that sums population over 1km2 pixels for each municipio
for y in ["2005","2010","2015","2020"] :

    #Load grided population of the world data
    gpw = ee.Image(f'CIESIN/GPWv4/population-count/{y}')
    gpw_pop = gpw.select("population-count")
    
    #Excecute function 
    gpw_y_sum = municipios.map(lambda x: mun_temp(x, gpw_pop, ee.Reducer.sum()))

    #Save csv to drive
    task = ee.batch.Export.table.toDrive(collection=gpw_y_sum,
                                     folder="pop_tot", # Nombre de la carpeta
                                     description=f"population_tot_{y}") #Nombre del archivo
                                     #fileFormat="shp") # Tipo de archivo
    task.start()

#### Loop over function over years for baseline forest (2000)

In [15]:
#Loop that sums population over 1km2 pixels of forests (2000)
for y in ["2005","2010","2015","2020"] :
    
    cover = cover_in_years[0].selfMask()
    
    #Load grided population of the world data
    gpw_y = ee.Image(f'CIESIN/GPWv4/population-count/{y}').updateMask(cover)
    gpw_y_pop = gpw_y.select("population-count")
    gpw_y_sum = municipios.map(lambda x: mun_temp(x, gpw_y_pop, ee.Reducer.sum()))
    
    #Save csv to drive
    task = ee.batch.Export.table.toDrive(collection=gpw_y_sum,
                                     folder="pop_in_forests_2000", # Nombre de la carpeta
                                     description=f"population_forest_2000_{y}") # Nombre del archivo
                                     #fileFormat="shp") # Tipo de archivo
    task.start()